In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import analysis

## Test decomposition of $\Gamma_{\mu\nu} = \Pi_1 \Lambda^1_{\mu\nu} + \Pi_2\Lambda^2_{\mu\nu}$

In [2]:
jobid = 20401
file = '/Users/theoares/lqcd/npr_momfrac/analysis_output/mixing_job' + str(jobid) + '/testPi.h5'
f = h5py.File(file, 'r')
LL = [16, 16, 16, 48]

In [3]:
k_list = f['momenta'][()]
mom_list = [[2 * np.sin(np.pi * k[mu] / LL[mu]) for mu in range(4)] for k in k_list]
Gamma = f['Gamma'][()]
Pi11 = f['Pi11'][()]
Pi12 = f['Pi12'][()]
Zq = f['Zq'][()]

In [4]:
# Gamma should satisfy Gamma = Pi11 Lambda1 + Pi12 Lambda2
L1_irr, L2_irr = [], []
for p in mom_list:
    L1, L2 = analysis.Lambda1(p), analysis.Lambda2(p)
    L1_irr.append([
        (L1[2, 2] - L1[3, 3] ) / np.sqrt(2),
        (L1[0, 0] - L1[1, 1] ) / np.sqrt(2),
        (L1[0, 0] + L1[1, 1] - L1[2, 2] - L1[3, 3] ) / 2,
    ])
    L2_irr.append([
        (L2[2, 2] - L2[3, 3] ) / np.sqrt(2),
        (L2[0, 0] - L2[1, 1] ) / np.sqrt(2),
        (L2[0, 0] + L2[1, 1] - L2[2, 2] - L2[3, 3] ) / 2,
    ])
L1_irr, L2_irr = np.array(L1_irr), np.array(L2_irr)
PiLambda = np.einsum('pb,pzij->pzbij', Pi11, L1_irr) + np.einsum('pb,pzij->pzbij', Pi12, L2_irr)

In [ ]:
# TODO see what happens when you decompose the other Born term into a Lambda1 and Lambda 2 piece. What should we expect?

In [5]:
for kidx, k in enumerate(k_list):
    print(k)
    for i in range(3):
        print('Irrep ' + str(i) + ', boot 0')
        Δ = Gamma[kidx, i] - PiLambda[kidx, i]
        print('Delta')
        print(np.mean(Δ, axis = 0))
        print(np.std(Δ, axis = 0, dtype = np.complex64))
        print('Gamma')
        print(np.mean(Gamma[kidx, i], axis = 0))
        print(np.std(Gamma[kidx, i], axis = 0))
        print('Lambda 1')
        print(L1_irr[kidx, i])
        print('Lambda 2')
        print(L2_irr[kidx, i])

[1 0 1 0]
Irrep 0, boot 0
Delta
[[-0.0265151 -0.00153586j  0.0003335 +0.00231004j  0.00707319+0.05544929j
   0.00338438-0.00178296j]
 [-0.00413867+0.00313356j -0.02380908+0.00064281j -0.00247637+0.0035757j
  -0.00403797+0.05498193j]
 [-0.00792312+0.05621838j -0.00242049+0.00134913j -0.02818474-0.00170918j
   0.0005519 +0.00099176j]
 [-0.00248937-0.00181301j -0.0026592 +0.05417821j  0.00486422-0.00356812j
  -0.02385108+0.00041042j]]
[[0.00453304+0.j 0.00373063+0.j 0.00476293+0.j 0.00266708+0.j]
 [0.00373804+0.j 0.00416735+0.j 0.0029966 +0.j 0.00483826+0.j]
 [0.00494016+0.j 0.00295397+0.j 0.00429984+0.j 0.00345502+0.j]
 [0.0031805 +0.j 0.00536739+0.j 0.00397404+0.j 0.0047729 +0.j]]
Gamma
[[-2.6515095e-02-0.00153586j  3.3350440e-04+0.00231004j
   3.4579611e-01+0.0576428j   3.6815407e-03-0.00161912j]
 [-4.1386676e-03+0.00313356j -2.3809079e-02+0.00064281j
  -2.1792094e-03+0.00373953j -3.4276092e-01+0.05278843j]
 [-3.4664604e-01+0.05402487j -2.7176503e-03+0.0011853j
  -2.8184740e-02-0.00170

In [39]:
# p = [3, 1, 0, 5]
# x should be a tensor of Dirac matrices, x = x[mu, nu, i, j]
# Have tested this block of code with x being a linear combination of Lambda1(p) and Lambda2(p)
# First try x as the numerical ΓB, then try it with the analytical one. 
# NUMERICAL ΓB SHOULD BE LIVING IN THE SPACE SPANNED BY {Λ1, Λ2}. 
# MAKE SURE THAT ANALYTICAL BORN TERM LIVES IN THAT SPACE TOO

k = [2, 1, 4, 0]
kstr = analysis.plist_to_string(k)
p = analysis.to_lattice_momentum(k)

y = 8 * analysis.Lambda1(p) +  3 * analysis.Lambda2(p)# + np.random.rand(4, 4, 4, 4)
v1 = np.array([
    analysis.inner(analysis.Lambda1(p), y),
    analysis.inner(analysis.Lambda2(p), y)
])
print('v1 is:')
print(v1)
Pi_i = np.dot(analysis.A_inv_ab(p), v1)
print('Decomposition:')
print(Pi_i)

print('Max difference between x and decomposition')
δy = y - (Pi_i[0] * analysis.Lambda1(p) + Pi_i[1] * analysis.Lambda2(p))
print(np.max(np.abs(δy)))

v1 is:
[-306.78022214+0.j -159.56789009+0.j]
Decomposition:
[8.+0.j 3.+0.j]
Max difference between x and decomposition
3.5596458096434965e-15


In [ ]:
ΓB[kstr]

In [ ]:
ΓB_irr1, ΓB_inv_irr1 = analysis.born_term_irreps(k_list)

## Playing around

In [37]:
# TODO should this be about equal to Zq? What should a free field Z look like?
# Play around with tr{Gamma Gamma_B^-1}
# Now decompose the free field solution into Pi_A and Pi_V
# This one should live in the space spanned by Lambda1 and Lambda2
ΓB = {analysis.plist_to_string(k) : np.zeros((4, 4, 4, 4), dtype = np.complex64) for k in k_list}
k_list = [[1, 1, 1, 1]]

for mu in range(4):
    #tmp, ΓB_inv = analysis.born_term_numerical(mu, momenta = k_list)
    tmp = analysis.born_term_numerical(mu, momenta = k_list)
#     tmp = analysis.born_term(mu, momenta = k_list)
    for k in k_list:
        kstr = analysis.plist_to_string(k)
        ΓB[kstr][mu, mu] = tmp[kstr]

ΓB_irr = [{}, {}, {}]
for kstr in ΓB.keys():
    ΓB_irr[0][kstr] = (ΓB[kstr][2, 2] - ΓB[kstr][3, 3]) / np.sqrt(2)
    ΓB_irr[1][kstr] = (ΓB[kstr][0, 0] - ΓB[kstr][1, 1]) / np.sqrt(2)
    ΓB_irr[2][kstr] = (ΓB[kstr][0, 0] + ΓB[kstr][1, 1] - ΓB[kstr][2, 2] - ΓB[kstr][3, 3]) / 2
# ΓB_irr, ΓB_inv_irr = analysis.born_term_irreps(k_list)
for idx, k in enumerate(k_list):
    kstr = analysis.plist_to_string(k)
    plat = analysis.to_lattice_momentum(k)
    L1, L2 = analysis.Lambda1(plat), analysis.Lambda2(plat)
    v = np.array([
        sum([np.einsum('ij,ji', analysis.tau13_irrep(L1, n), ΓB_irr[n][kstr]) for n in range(3)]),    # TODO if we divide this by 3 then we get close
        sum([np.einsum('ij,ji', analysis.tau13_irrep(L2, n), ΓB_irr[n][kstr]) for n in range(3)]),
    ])
    print('v is: ')
    print(v)
    Ainv = analysis.A_inv_ab(plat)
    Pi1, Pi2 = Ainv.dot(v)
    print('Pi1: ' + str(Pi1) + ', Pi2: ' + str(Pi2))
    print('Difference:')
    δ = ΓB[kstr] - (Pi1 * L1 + Pi2 * L2)    # make sure to only get the diagonal
    #print(δ)
    δ_max = max([np.max(np.abs(δ[mu, mu])) for mu in range(4)])
    print(δ_max)
#     print(np.max(np.abs(δ)))

v is: 
[-2.97091351+0.j -0.15695847+0.j]
Pi1: (1.9999999970876687+0j), Pi2: (-1.2556965245380525e-08+0j)
Difference:
0.27589938244525714


In [31]:
print(ΓB[kstr][0, 1])
print(2 * L1[0, 1])
# print(ΓB[kstr] - 2 * L1)

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[ 0.        +0.j          0.        +0.j          0.        +0.j
   0.39018065+0.39018065j]
 [ 0.        +0.j          0.        +0.j          0.39018065-0.39018065j
   0.        +0.j        ]
 [ 0.        +0.j         -0.39018065-0.39018065j  0.        +0.j
   0.        +0.j        ]
 [-0.39018065+0.39018065j  0.        +0.j          0.        +0.j
   0.        +0.j        ]]


In [ ]:
# we want Delta to be 0
Δ = []
for i, k in enumerate(k_list):
    kstr = analysis.plist_to_string(k)
    Δ.append(decomp[i] - ΓB[kstr])    # should be equal to Γ_{μνab}

# currently gets 0 in the right places, but has an issue with the normalization
print('Delta')
print(Δ[0][0, 0])
print('Decomposition')
print(decomp[0][0, 0])
print('ΓB')
print(ΓB['p1010'][0, 0])

In [ ]:
# Test out actual Γ
fpi = h5py.File('/Users/theoares/lqcd/npr_momfrac/analysis_output/mixing_job22454/Pi_subset.h5', 'r')
kpi_list = fpi['momenta'][()]
ppi_list = [[2 * np.sin(np.pi * k[mu] / LL[mu]) for mu in range(4)] for k in kpi_list]
Γ = fpi['Gamma'][()]
Pi1a = fpi['Pi11'][()]
Pi2a = fpi['Pi12'][()]
n_boot = 50

In [ ]:
# Γmunu = {analysis.plist_to_string(k) : np.zeros((50, 4, 4, 4, 4), dtype = np.complex64) for k in kpi_list}
Γmunu = {analysis.plist_to_string(k) : np.zeros((4, 4, 4, 4), dtype = np.complex64) for k in kpi_list}
for mu in range(4):
    for idx, k in enumerate(kpi_list):
        kstr = analysis.plist_to_string(k)
        #Γmunu[kstr][:, mu, mu] = Γ[idx, mu]
        Γmunu[kstr][mu, mu] = Γ[idx, mu, 0]

In [ ]:
#     k = [2, 1, 4, 0]
for idx, k in enumerate(kpi_list):
    print(k)
    kstr = analysis.plist_to_string(k)
    p = analysis.to_lattice_momentum(k)

    # x = 8 * analysis.Lambda1(p) +  3 * analysis.Lambda2(p) + np.random.rand(4, 4, 4, 4)
    x = Γmunu[kstr]
    v1 = np.array([
        analysis.inner(analysis.Lambda1(p), x),
        analysis.inner(analysis.Lambda2(p), x)
    ])
#     print('v1 is:')
#     print(v1)
    # TODO compare this output to 20401 -- It looks the same. Seems like outliers in 20401 just have particularly high errorbars 
    Pi_i = np.dot(analysis.A_inv_ab(p), v1)
    print('Decomposition:')
    print(Pi_i)

    print('Max difference between x and decomposition')
    δx = x - (Pi_i[0] * analysis.Lambda1(p) + Pi_i[1] * analysis.Lambda2(p))
    print(np.max(np.abs(δx)))

## Test free fields solution for vector and axial currents

In [ ]:
# Examine free field amputated Green's function for currents. Want ΓV[mu] = gamma_mu, ΓA[mu] = gamma_mu * gamma_5
fc = h5py.File('/Users/theoares/lqcd/npr_momfrac/analysis_output/free_field_currents/Z.h5', 'r')
kc_list = fc['momenta'][()]
pc_list = [[2 * np.sin(np.pi * k[mu] / LL[mu]) for mu in range(4)] for k in kc_list]
ΓV = fc['GammaV'][()]
ΓA = fc['GammaA'][()]

# Set noise to 0 (could also average it if less lazy)
ϵ = 1e-10
ΓV[np.abs(ΓV) < ϵ] = 0
ΓA[np.abs(ΓA) < ϵ] = 0

# strip off extra indices
ΓV = np.einsum('mpaiaj->mpij', ΓV[:, :, 0, :, :, :, :]) / 3
ΓA = np.einsum('mpaiaj->mpij', ΓA[:, :, 0, :, :, :, :]) / 3

In [ ]:
# Check that ΓV equals gamma_mu. Note that ΓV should be independent of p, so we can average over p and look at the std to make sure.
avgV = np.mean(ΓV, axis = 1)
sigmaV = np.std(ΓV, axis = 1)
print('Vector current')
for mu in range(4):
    print('Average for mu = ' + str(mu))
    print(avgV[mu])
    δV = avgV[mu] - analysis.gamma[mu]
    print('Difference with γ[μ = ' + str(mu) + ']')
    print(δV)
    print('Max difference:')
    print(np.max(np.abs(δV)))
    print('Error for mu = ' + str(mu))
    print(sigmaV[mu])

In [ ]:
# Check that ΓV equals gamma_mu. Note that ΓV should be independent of p, so we can average over p and look at the std to make sure.
avgA = np.mean(ΓA, axis = 1)
sigmaA = np.std(ΓA, axis = 1)
print('Axial current')
for mu in range(4):
    print('Average for mu = ' + str(mu))
    print(avgA[mu])
    δA = avgA[mu] - np.dot(analysis.gamma[mu], analysis.gamma5)
    print('Difference with $γ[μ = ' + str(mu) + '] * γ_5$')
    print(δA)
    print('Max difference:')
    print(np.max(np.abs(δA)))
    print('Error for mu = ' + str(mu))
    print(sigmaA[mu])

## Test sum of point sources vs sum of wall sources

In [ ]:
# jobid = 21209
V = (16 ** 3) * 48
jobid = 21695
file = '/Users/theoares/lqcd/npr_momfrac/output/pt_wall_test_' + str(jobid) + '/job' + str(jobid) + '.h5'
f = h5py.File(file, 'r')

In [ ]:
mom_list = np.array([[i, i, i, i] for i in range(1, 6)] + [[2, 2, 2, 4], [1, 1, 1, 4], [1, 1, 1, 2]])
S_sub_op, S_op, S_sink = [], [], []
for p in mom_list:
    pstr = analysis.plist_to_string(p)
    #S_sub_op.append(f['prop_op_sub'][pstr][()])
    S_op.append(f['prop_op'][pstr][()])
    S_sink.append(f['prop_sink'][pstr][()])
print('Computing difference between sum of point sources and momentum wall source.')
for i in range(len(S_op)):
    print(np.max(S_op[i] - S_sink[i]))

In [ ]:
Nlist = [-1, -2, 1, 2, 5, 10, 20, 30, 40, 50]
Zq = {}
for N in Nlist:
    if N == -1:
        key = 'prop_sink'
    elif N == -2:
        key = 'prop_op'
    else:
        key = 'prop_op_sub/N' + str(N)
    props = {}
    Zq[N] = []
    for p in mom_list:
        pstr = analysis.plist_to_string(p)
#         props[pstr] = np.zeros((1, 4, 4, 3, 3), dtype = np.complex64)
        props[pstr] = np.expand_dims(np.einsum('ijab->aibj', f[key + '/' + pstr][()]), axis = 0)
#         props[pstr][0] = np.einsum('ijab->iajb', f[key + '/' + pstr][()])
#         props[pstr][0] = f[key + '/' + pstr][()]
    props_boot = analysis.bootstrap(props)
    props_inv = analysis.invert_prop(props_boot, 1)
    Zqboot = analysis.quark_renorm(props_inv)
    Zq[N] = [np.mean(Zqboot[analysis.plist_to_string(p)]) for p in mom_list]

In [ ]:
print(Zq[-1])
print(Zq[-2])

In [ ]:
p_squared = [analysis.square(analysis.to_lattice_momentum(x)) for x in mom_list]
plt.figure()
for N in Nlist:
    if N == -1:
        label = 'wall'
    elif N == -2:
        label = 'all points'
    else:
        label = 'N = ' + str(N)
    plt.scatter(p_squared, np.real(Zq[N]) / V, label = label)    # psquared vs all Zq
    plt.legend()
plt.show()

In [ ]:
# Phiala's point source code which is correct
f = h5py.File('/Users/theoares/lqcd/npr_momfrac/phiala_code/analysis_output/quarkNPR_cl3_16_48_b6p1_m0p2450.h5', 'r')
k0_list, mom0_list, p0_squared, Zq0, Zq0_μ, Zq0_σ = [], [], [], [], [], []
for i in range(0, 5):
    for j in range(0, 5):
        for l in range(0, 5):
            for m in range(0, 5):
                k = [i, j, l, m]
                p = analysis.to_lattice_momentum(k)
                k0_list.append(k)
                mom0_list.append(p)
                p0_squared.append(analysis.square(p))
                str_idx = str(i) + str(j) + str(l) + str(m)
                Z = f[str_idx + '/Zq'][()]
                Zq0.append(Z)
                Zq0_μ.append(np.mean(Z))
                Zq0_σ.append(np.std(Z))

In [ ]:
# My point source code which is not correct
f2 = h5py.File('/Users/theoares/lqcd/npr_momfrac/phiala_code/analysis_output/my_Zq_analysis.h5', 'r')
Zq2 = f2['Zq'][()]
k2_list = f2['momenta'][()]
mom2_list = [analysis.to_lattice_momentum(k) for k in k2_list]
p2_squared = [analysis.square(p) for p in mom2_list]
Zq2_μ = np.mean(Zq2, axis = 1)
Zq2_σ = np.std(Zq2, axis = 1)

In [ ]:
# My wall source code which should also be correct
wall_src_file = '/Users/theoares/lqcd/npr_momfrac/analysis_output/jobZq19214/Zq.h5'
wk, wp, wZq = analysis.load_Zq(wall_src_file)
wp_squared = np.array([analysis.square(p) for p in wp])
Zq_wall = np.mean(wZq, axis = 1)
σZq_wall = np.std(wZq, axis = 1)

In [ ]:
plt.figure()
plt.errorbar(p0_squared, np.real(Zq0_μ), yerr = np.real(Zq0_σ), c = 'r', fmt = 'o', label = 'phiala analysis output')
plt.errorbar(p2_squared, np.real(Zq2_μ), yerr = np.real(Zq2_σ), c = 'b', fmt = 'o', label = 'my analysis output')
plt.errorbar(wp_squared, np.real(Zq_wall), np.real(σZq_wall), c = 'g', fmt = 'o', label = 'wall')
plt.legend()
plt.show()

In [ ]:
k_list, mom_list, p_squared, Zq = [], [], [], []
for key in f:
    k = f[key + '/mom'][()]
    p = analysis.to_lattice_momentum(k)
    k_list.append(k)
    mom_list.append(p)
    p_squared.append(analysis.square(p))
    Zq.append(f[key + '/Zq'][()])

In [ ]:
f['-1-1-1-1/Zq']

In [ ]:
for x in f:
    print(f[x + '/mom'][()])

In [ ]:
Zq = []
for k in k_list:
    f[]
f.keys()